# PostgreSQL 16.6+

## 0) 前提

* エンジン: **PostgreSQL 16.6+**
* 並び順: 任意（`ORDER BY` なし）
* `NOT IN` 回避（本問では不使用）
* 判定は **ID 基準**、表示は仕様どおり `actor_id, director_id`

## 1) 問題

* `同一の (actor_id, director_id) が3回以上出現する協働ペアを抽出せよ。`
* 入力: `ActorDirector(actor_id int, director_id int, "timestamp" int PRIMARY KEY)`
* 出力: `actor_id, director_id`（重複なし・順不同）

---

## 2) 最適解（単一クエリ）

> PostgreSQL でも素直に **ウィンドウ + 外側で重複排除** で書けます（要件充足）。ただし実務では後述の **`GROUP BY` 案が最速**になりがち。

```sql
WITH pre AS (
  SELECT actor_id, director_id
  FROM ActorDirector
),
win AS (
  SELECT
    actor_id,
    director_id,
    COUNT(*) OVER (PARTITION BY actor_id, director_id) AS coop_cnt
  FROM pre
)
SELECT DISTINCT
  actor_id,
  director_id
FROM win
WHERE coop_cnt >= 3;

Runtime
314
ms
Beats
44.19%

```

### 代替（推奨：集約一発）

> PostgreSQL のプランナは集約に強いので、こちらが概ね速いです。

```sql
SELECT
  actor_id,
  director_id
FROM ActorDirector
GROUP BY actor_id, director_id
HAVING COUNT(*) >= 3;

Runtime
295
ms
Beats
69.28%

```

> 結果は同じ。**読みやすく、余計な行膨張もない**ため、まずはこれで。

---

## 3) 要点解説

* **最小十分条件**は「ペア単位の件数 ≥ 3」だけ。
  ウィンドウを使う場合も `COUNT(*) OVER (PARTITION BY ...)` で総回数を付与してから閾値抽出。
* **集合演算としては `GROUP BY ... HAVING COUNT(*) >= 3` が最短ルート**。
  ウィンドウ版は内側で行を複製するため I/O が増えやすい。
* **列名** `timestamp` は予約語ではないものの関数と紛らうので、DDL 上はダブルクォートが安全（上の例は既存前提）。

---

## 4) 計算量（概算）

* ウィンドウ: **O(Σ n_g log n_g)**（パーティション内の並べ替え/バッファリング次第）
* 集約（推奨クエリ）: **O(N)**〜**O(N log N)**（HashAggregate or GroupAggregate）

---

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 テーブル]
  B[ペアごとに件数を算出]
  C[しきい値 3 以上を抽出]
  D[出力 actor_id director_id]
  A --> B
  B --> C
  C --> D
```

---

### 実運用メモ（速度チューニング）

* **インデックス**
  集約を「並び順で一発巻き上げ」できる余地を作ります。

  ```sql
  CREATE INDEX ON ActorDirector (actor_id, director_id);
  ```

  * これで `GroupAggregate`（インデックス順）か `HashAggregate` のいずれか良い方を選びやすくなります。
* **統計**

  ```sql
  ANALYZE ActorDirector;
  ```

  古い統計だとフルスキャンを選びがち。
* **ワークメモリ**
  大きめのグループ数で `HashAggregate` が溢れる場合はセッション単位で調整：

  ```sql
  SET work_mem = '128MB';  -- ワークロードに合わせて
  ```
* **確認**
  `EXPLAIN (ANALYZE, BUFFERS)` で `HashAggregate` か `GroupAggregate`、`Rows Removed by Filter`、`Shared Hit/Read` を確認。
  さらに縮める必要があれば、取り込みパイプラインで**サマリテーブル**（actor_id, director_id ごとの coop_cnt）をバッチ更新するのが王道です。

結論：**`GROUP BY ... HAVING COUNT(*) >= 3` が最短コース**です。ここから先は**物理設計と実行計画**で詰めます。実装はそのまま、周辺を最適化しましょう。

---

## 推奨チューニング手順（優先度順）

### 1) 複合インデックス（最重要）

ペアの集約を**インデックス順で一発**に寄せます。

```sql
-- 既存トラフィックがあるなら同時作成を推奨
CREATE INDEX CONCURRENTLY IF NOT EXISTS idx_actor_director
  ON ActorDirector (actor_id, director_id);
```

効果：

* `GroupAggregate` が **Index Only/Index Scan** ベースになりやすい
* 返す列がキーのみのため **カバリング**（VMが立てば実質 Index Only Scan）

### 2) 統計・可視化（Index Only Scan を狙う）

```sql
ANALYZE ActorDirector;         -- 統計更新（必須）
VACUUM (ANALYZE) ActorDirector; -- 可視マップ(VM)を立てて Index Only Scan 率UP
```

* 可視マップが育つと「テーブル読み」回数が下がります。

### 3) 充分ならウィンドウ版は封印

ウィンドウは行を膨らませるので I/O 増。最速の本命は下記。

```sql
SELECT actor_id, director_id
FROM ActorDirector
GROUP BY actor_id, director_id
HAVING COUNT(*) >= 3;
```

### 4) HashAggregate のスピル対策（必要時のみ）

`EXPLAIN (ANALYZE, BUFFERS)` で HashAggregate がディスクに溢れていたら一時的に：

```sql
SET work_mem = '128MB';  -- ワークロードに合わせ調整
-- 比較用（悪化する場合もあるので計測前提）
SET enable_hashagg = on;      -- 既定
-- or
SET enable_hashagg = off;     -- GroupAggregate に寄せる比較用
```

* **spilling**（Disk: ～MB）が消えるか、`GroupAggregate` で高速化するかを計測。

### 5) 並列実行の活用（テーブルが大きい場合）

```sql
SET max_parallel_workers_per_gather = 2;  -- 環境許容量に応じて
```

* `Parallel Index/Seq Scan + Parallel Hash/Group Aggregate` を取りやすくなります。

### 6) 物理配置の最適化（更新が少ないなら）

```sql
-- インデックス順に再配置（ダウンタイム許容時）
CLUSTER ActorDirector USING idx_actor_director;
-- オンラインなら pg_repack も選択肢
```

* `(actor_id, director_id)` で連続化 → キャッシュ効率改善。

---

## EXPLAIN チェックリスト（理想像）

* `GroupAggregate` or `HashAggregate` がトップ
* `key=idx_actor_director` を使っている（`Index Scan/Index Only Scan`）
* `Extra`/出力に **Disk: 0**（スピルなし）
* `Shared Read` が小さく、`Hit` が多い
* `Rows Removed by Filter` が少ない（余計な読みが少ない）

---

## 規模がさらに大きい場合の構造策

### サマリテーブル（マテビュー代替）

高頻度の照会なら恒常的に最速です。

```sql
-- 初期構築
CREATE TABLE CoopSummary AS
SELECT actor_id, director_id, COUNT(*) AS coop_cnt
FROM ActorDirector
GROUP BY actor_id, director_id;

CREATE UNIQUE INDEX IF NOT EXISTS ux_coop ON CoopSummary(actor_id, director_id);

-- 照会は
SELECT actor_id, director_id
FROM CoopSummary
WHERE coop_cnt >= 3;
```

更新はバッチで増分反映（新着分を集計→`INSERT ... ON CONFLICT ... DO UPDATE`）。

---

## 期待できる改善幅（目安）

* **インデックス + 統計更新**だけで、350ms → **100ms台**は十分現実的（データ量・I/O次第）。
* さらに **Index Only Scan** や **並列化**、**物理配置**が噛むと二桁msに入るケースも。

---

## まとめ（実行順）

1. `CREATE INDEX CONCURRENTLY ON ActorDirector(actor_id, director_id);`
2. `VACUUM (ANALYZE) ActorDirector;`
3. 本命クエリは `GROUP BY ... HAVING COUNT(*) >= 3`
4. まだ遅ければ `work_mem` / `enable_hashagg` を計測調整
5. それでも重い常用クエリなら **サマリテーブル**化

この順で叩けば、現状（295ms/314ms）から**まだ削れます**。

